#### Progetto di corso per APPLICAZIONI DELL'INTELLIGENZA ARTIFICIALE (AA 2024-2025)
#### Stud: Marzio Della Bosca


Jupyter Notebook con la funzione di estrarre le feature, tramite catch22 e tsfel, dai dati contenuti nel dataset [Heterogeneity Activity Recognition](https://archive.ics.uci.edu/dataset/344/heterogeneity+activity+recognition)

In [1]:
import tsfel
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import time
import os
from aeon.transformations.collection.feature_based import Catch22

# Gt o label: Biking, Sitting, Standing, Walking, Stair Up, Stair down

In [2]:
accelerometer_data = pd.read_csv('./HHAR_data/Phones_accelerometer_hhar.csv')
gyroscope_data = pd.read_csv('./HHAR_data/Phones_gyroscope_hhar.csv')

print(f"Numero Dati Accelerometro: {len(accelerometer_data)}, Head:")
print(accelerometer_data.head(4))

print(f"\nNumero Dati Giroscopio: {len(gyroscope_data)}, Head:")
print(gyroscope_data.head(4))

Numero Dati Accelerometro: 13062475, Head:
   Index   Arrival_Time        Creation_Time         x         y         z  \
0      0  1424696633908  1424696631913248572 -5.958191  0.688065  8.135345   
1      1  1424696633909  1424696631918283972 -5.952240  0.670212  8.136536   
2      2  1424696633918  1424696631923288855 -5.995087  0.653549  8.204376   
3      3  1424696633919  1424696631928385290 -5.942718  0.676163  8.128204   

  User   Model    Device     gt  
0    a  nexus4  nexus4_1  stand  
1    a  nexus4  nexus4_1  stand  
2    a  nexus4  nexus4_1  stand  
3    a  nexus4  nexus4_1  stand  

Numero Dati Giroscopio: 13932632, Head:
   Index   Arrival_Time        Creation_Time         x         y         z  \
0      0  1424696633909  1424696631914042029  0.013748 -0.000626 -0.023376   
1      1  1424696633909  1424696631919046912  0.014816 -0.001694 -0.022308   
2      2  1424696633918  1424696631924051794  0.015884 -0.001694 -0.021240   
3      3  1424696633919  142469663192911771

In [19]:
def window_list(data_list, freq, window_seconds=2):                 # finestro a 2 secondi in maniera dinamica date le diverse frequenze di campionamento
    if freq <= 0 or not isinstance(data_list, list):
        return []
    window_size = int(freq * window_seconds)
    #print(freq)
    #print(window_size)
    if window_size == 0:
        return []
    return [data_list[i:i+window_size] 
            for i in range(0, len(data_list), window_size) 
            if len(data_list[i:i+window_size]) == window_size]

rows = []

def catch22_time_series_features_extractor(time_series):
    catch22 = Catch22()
    startTime = time.time()

    # Numero di feature Catch22 per canale (prendo il primo segnale x_acc)
    test_features = catch22.fit_transform([np.array(time_series.iloc[0]['x_acc'])])[0]
    n_feat = len(test_features)
    n_channels = 6  # x_acc, y_acc, z_acc, x_gyr, y_gyr, z_gyr

    # Preallocazione array 3D (serie, canali, feature)
    features_3d = np.zeros((len(time_series), n_channels, n_feat))
    activities = []

    channels = ['x_acc', 'y_acc', 'z_acc', 'x_gyr', 'y_gyr', 'z_gyr']

    for i, (_, series) in enumerate(tqdm(time_series.iterrows(), total=len(time_series), desc="Estrazione Catch22")):
        for j, ch in enumerate(channels):
            signal = np.array(series[ch])
            features_3d[i, j, :] = catch22.fit_transform([signal])[0]
        activities.append(series['activity'])

    activities_array = np.array(activities)

    print(f"Tempo di esecuzione per l'estrazione delle feature catch22: {time.time() - startTime:.2f} secondi")
    return features_3d, activities_array


def tsfel_time_series_features_extractor_dual(cfg_stat, cfg_temp, time_series):
    timeStart = time.time()

    # Determina il numero di feature per configurazione e canale usando la prima serie e primo canale
    test_feats_stat = tsfel.time_series_features_extractor(cfg_stat, pd.DataFrame(np.array(time_series.iloc[0]['x_acc'])), verbose=0)
    n_feat_stat = test_feats_stat.shape[1]

    test_feats_temp = tsfel.time_series_features_extractor(cfg_temp, pd.DataFrame(np.array(time_series.iloc[0]['x_acc'])), verbose=0)
    n_feat_temp = test_feats_temp.shape[1]

    n_channels = 6  # x_acc, y_acc, z_acc, x_gyr, y_gyr, z_gyr

    # Prealloca array per feature (stat + temp)
    features_3d = np.zeros((len(time_series), n_channels, n_feat_stat + n_feat_temp))
    activities = []

    channels = ['x_acc', 'y_acc', 'z_acc', 'x_gyr', 'y_gyr', 'z_gyr']

    for i, (_, series) in enumerate(tqdm(time_series.iterrows(), total=len(time_series), desc="Estrazione TSFEL")):
        for j, ch in enumerate(channels):
            signal = pd.DataFrame(np.array(series[ch]))  # Serie temporale canale
            feats_stat = tsfel.time_series_features_extractor(cfg_stat, signal, verbose=0, n_jobs=-1)
            feats_temp = tsfel.time_series_features_extractor(cfg_temp, signal, verbose=0, n_jobs=-1)
            # Concateno le feature stat + temp
            combined_feats = np.concatenate((feats_stat.values.flatten(), feats_temp.values.flatten()))
            features_3d[i, j, :] = combined_feats
        activities.append(series['activity'])

    print(f"Tempo di esecuzione per l'estrazione delle feature tsfel 3D: {time.time() - timeStart:.2f} secondi")

    activities_array = np.array(activities)
    return features_3d, activities_array

In [4]:
# Anche se diverse prove mostrano sia in head che in tail che i creation time dei dati di giroscopio e accelerometro sembrano combaciare si nota un'importante mancanza di 
# campioni dal giroscopio, inoltre sono stati usati diversi cellulari a diverse frequenze di campionamento. Raggruppando per 'User', 'Device' e 'Gt' riesco ad ottenere
# delle prove singole o comunque una sequenza di prove con stessa label e stesso dispositivo e tramite numero di campioni e Creation time di inizio e fine riesco a calcolare
# in maniera dinamica le frequenze di campionamento e riuscire a mantenere un finestramento di 2 secondi.

# estraggo solo le colonne necessarie per ciascun dataset
acc_data = accelerometer_data[['User', 'Device', 'gt', 'Creation_Time', 'x', 'y', 'z']].copy()
gyro_data = gyroscope_data[['User', 'Device', 'gt', 'Creation_Time', 'x', 'y', 'z']].copy()

# raggruppo per User, Device, gt e aggrego dati in liste
acc_grouped = acc_data.groupby(['User', 'Device', 'gt']).agg({
    'Creation_Time': list,
    'x': list,
    'y': list,
    'z': list
}).reset_index()

gyro_grouped = gyro_data.groupby(['User', 'Device', 'gt']).agg({
    'Creation_Time': list,
    'x': list,
    'y': list,
    'z': list
}).reset_index()

In [ ]:
print(acc_grouped.shape)
print(gyro_grouped.shape)

print(acc_grouped.head(4))

In [6]:
# Set delle triple chiave (User, Device, gt) per accelerometro e giroscopio, aggrego i dati da acc e gyr
acc_keys = set(acc_grouped.apply(lambda row: (row['User'], row['Device'], row['gt']), axis=1))
gyro_keys = set(gyro_grouped.apply(lambda row: (row['User'], row['Device'], row['gt']), axis=1))

# Differenze
only_in_acc = acc_keys - gyro_keys  # combinazioni presenti solo in acc
only_in_gyro = gyro_keys - acc_keys # combinazioni presenti solo in gyro
common_keys = acc_keys & gyro_keys   # combinazioni presenti in entrambi

print(f"Combinazioni solo in accelerometro: {len(only_in_acc)}")
print(f"Combinazioni solo in giroscopio: {len(only_in_gyro)}")
print(f"Combinazioni comuni: {len(common_keys)}")

Combinazioni solo in accelerometro: 108
Combinazioni solo in giroscopio: 1
Combinazioni comuni: 307


In [7]:
# Definisco una funzione che verifica se la tupla (User, Device, gt) di una riga è in common_keys
def is_common(row):
    return (row['User'], row['Device'], row['gt']) in common_keys

# Filtra acc_grouped mantenendo solo le righe con combinazioni in common_keys
acc_filtered = acc_grouped[acc_grouped.apply(is_common, axis=1)].reset_index(drop=True)

# Filtra gyro_grouped allo stesso modo
gyro_filtered = gyro_grouped[gyro_grouped.apply(is_common, axis=1)].reset_index(drop=True)

print(acc_filtered.shape)
print(gyro_filtered.shape)

(307, 7)
(307, 7)


In [8]:
# o colonna 'key' con la tupla (User, Device, gt), solo per controllo
acc_filtered['key'] = list(zip(acc_filtered['User'], acc_filtered['Device'], acc_filtered['gt']))
gyro_filtered['key'] = list(zip(gyro_filtered['User'], gyro_filtered['Device'], gyro_filtered['gt']))

# Controlla se sono uguali (ordine e valori)
print(f"\nLe prime {(len(acc_filtered)+len(gyro_filtered))//2} chiavi sono uguali?: {(acc_filtered['key'].head(len(acc_filtered)) == gyro_filtered['key'].head(len(gyro_filtered))).all()}")



Le prime 307 chiavi sono uguali?: True


In [9]:
acc_filtered = acc_filtered.drop(columns=['key'])
gyro_filtered = gyro_filtered.drop(columns=['key'])

# Calcola (max - min) per ogni lista di timestamps (registrazioni)
acc_filtered['Duration_acc'] = acc_filtered['Creation_Time'].apply(
    lambda lst: (max(pd.to_datetime(lst)) - min(pd.to_datetime(lst))) 
                if isinstance(lst, list) and lst else pd.Timedelta(0)
)
gyro_filtered['Duration_gyr'] = gyro_filtered['Creation_Time'].apply(
    lambda lst: (max(pd.to_datetime(lst)) - min(pd.to_datetime(lst))) 
                if isinstance(lst, list) and lst else pd.Timedelta(0)
)

acc_filtered = acc_filtered.drop(columns=['Creation_Time'])
gyro_filtered = gyro_filtered.drop(columns=['Creation_Time'])

gyro_data_only = gyro_filtered[['Duration_gyr', 'x', 'y', 'z']].rename(columns={
    'x': 'x_gyr', 'y': 'y_gyr', 'z': 'z_gyr'
})
acc_data_only = acc_filtered[['User', 'Device', 'gt', 'Duration_acc', 'x', 'y', 'z']].rename(columns={
    'x': 'x_acc', 'y': 'y_acc', 'z': 'z_acc'
})

merged_df = pd.concat(
    [acc_data_only.reset_index(drop=True), gyro_data_only.reset_index(drop=True)],
    axis=1
)

In [10]:
print(merged_df.shape)
print(merged_df.head())

(307, 11)
  User    Device          gt              Duration_acc  \
0    a  nexus4_1        bike 0 days 00:04:59.734857830   
1    a  nexus4_1         sit 0 days 00:05:07.898901008   
2    a  nexus4_1  stairsdown 0 days 00:07:52.720734654   
3    a  nexus4_1    stairsup 0 days 00:07:14.813174234   
4    a  nexus4_1       stand 1 days 12:27:10.086740992   

                                               x_acc  \
0  [-7.291198700000001, -7.216216999999999, -7.21...   
1  [-0.44882202, -0.42739868, -0.41430664, -0.382...   
2  [-5.9950867, -6.33667, -6.817505000000001, -7....   
3  [-4.3323975, -4.450226, -4.516876, -4.552582, ...   
4  [-5.958191, -5.95224, -5.9950867, -5.9427185, ...   

                                               y_acc  \
0  [2.3412323, 2.2710114, 2.2710114, 2.2281647, 2...   
1  [-0.0962677, -0.07722473, -0.084365845, -0.097...   
2  [2.5614166000000003, 2.4066925, 2.2150726, 2.0...   
3  [-0.19029236, -0.04270935, 0.21913147, 0.45835...   
4  [0.6880646, 0.6702118

In [ ]:
merged_df['Duration_acc'] = merged_df['Duration_acc'].dt.total_seconds()
merged_df['Duration_gyr'] = merged_df['Duration_gyr'].dt.total_seconds()

print("Shape: ", merged_df.shape) 

# Filtro: mantiene righe con durata > 1 ora, probabilmente sono registrazioni in cui mancano timestamp rilevanti o diverse registrazioni fatte dallo stesso user in giorni diversi
long_sessions = merged_df[
    (merged_df['Duration_acc'] > 3600) | (merged_df['Duration_gyr'] > 3600)
]

print(long_sessions[['User', 'Device', 'gt', 'Duration_acc', 'Duration_gyr']])

# Tiene solo le righe con durata <= 1 ora sia per acc sia per gyr
merged_df = merged_df[
    (merged_df['Duration_acc'] <= 3600) & (merged_df['Duration_gyr'] <= 3600)
]

print("Shape senza elemento confuso: ", merged_df.shape)

Shape:  (307, 11)
  User    Device     gt   Duration_acc   Duration_gyr
4    a  nexus4_1  stand  131230.086741  131230.081645
Shape senza elemento confuso:  (306, 11)


In [13]:
merged_df['Freq_acc'] = merged_df.apply(                                    # calcolo la colonna delle frequenze (calcolate dinamicamente)
    lambda row: len(row['x_acc']) / row['Duration_acc']
    if row['Duration_acc'] > 0 else 0,
    axis=1
)

merged_df['Freq_gyr'] = merged_df.apply(
    lambda row: len(row['x_gyr']) / row['Duration_gyr']
    if row['Duration_gyr'] > 0 else 0,
    axis=1
)

freq_table = merged_df[['User', 'Device', 'gt', 'Freq_acc', 'Freq_gyr']]
print(freq_table)

    User    Device          gt    Freq_acc    Freq_gyr
0      a  nexus4_1        bike  122.361477  122.361477
1      a  nexus4_1         sit  194.719760  194.712145
2      a  nexus4_1  stairsdown   95.172470   95.191521
3      a  nexus4_1    stairsup   99.293220   99.279421
5      a  nexus4_1        walk  186.133483  186.144424
..   ...       ...         ...         ...         ...
302    i  s3mini_2         sit   60.401909   60.175103
303    i  s3mini_2  stairsdown   70.318361   70.456379
304    i  s3mini_2    stairsup   64.321748   63.859096
305    i  s3mini_2       stand   60.764675   60.578505
306    i  s3mini_2        walk   76.399430   76.297190

[306 rows x 5 columns]


In [ ]:
for idx, row in merged_df.iterrows():
    # calcolo finestre
    x_acc_windows = window_list(row['x_acc'], row['Freq_acc'], 2)
    y_acc_windows = window_list(row['y_acc'], row['Freq_acc'], 2)
    z_acc_windows = window_list(row['z_acc'], row['Freq_acc'], 2)

    x_gyr_windows = window_list(row['x_gyr'], row['Freq_gyr'], 2)
    y_gyr_windows = window_list(row['y_gyr'], row['Freq_gyr'], 2)
    z_gyr_windows = window_list(row['z_gyr'], row['Freq_gyr'], 2)

    # numero minimo di finestre tra tutti i canali per evitare mismatch
    n_windows = min(
        len(x_acc_windows), len(y_acc_windows), len(z_acc_windows),
        len(x_gyr_windows), len(y_gyr_windows), len(z_gyr_windows)
    )

    # genera nuove righe per ogni finestra
    for i in range(n_windows):
        rows.append({
            'gt': row['gt'],
            'x_acc': x_acc_windows[i],
            'y_acc': y_acc_windows[i],
            'z_acc': z_acc_windows[i],
            'x_gyr': x_gyr_windows[i],
            'y_gyr': y_gyr_windows[i],
            'z_gyr': z_gyr_windows[i],
        })

windowed_df = pd.DataFrame(rows)

print(windowed_df.shape)

(46787, 7)


In [16]:
windowed_df = windowed_df.rename(columns={'gt': 'activity'})
# Sposta la colonna 'activity' alla fine
cols = [col for col in windowed_df.columns if col != 'activity'] + ['activity']
windowed_df = windowed_df[cols]

print(windowed_df.columns)

Index(['x_acc', 'y_acc', 'z_acc', 'x_gyr', 'y_gyr', 'z_gyr', 'activity'], dtype='object')


In [20]:
X = windowed_df.copy()
y = windowed_df['activity']
X = X.drop('activity', axis=1)

print("Shape delle feature (X):", X.shape)
print("Shape delle etichette (y):", y.shape)

time_series = windowed_df.copy()

Shape delle feature (X): (46787, 6)
Shape delle etichette (y): (46787,)


In [21]:
X_catch22, activities = catch22_time_series_features_extractor(time_series)

Estrazione Catch22: 100%|██████████| 46787/46787 [04:15<00:00, 183.05it/s]

Tempo di esecuzione per l'estrazione delle feature catch22: 256.55 secondi


In [ ]:
print(f"Shape X_catch22: {X_catch22.shape}")
np.save('hhar_catch22.npy', X_catch22)

Shape X_catch22: (46787, 6, 22)


In [23]:
cfg_stat = tsfel.get_features_by_domain('statistical')
cfg_temp = tsfel.get_features_by_domain('temporal')

X_tsfel, activities = tsfel_time_series_features_extractor_dual(cfg_stat, cfg_temp, time_series)

Estrazione TSFEL: 100%|██████████| 46787/46787 [14:04<00:00, 55.41it/s]

Tempo di esecuzione per l'estrazione delle feature tsfel 3D: 844.36 secondi


In [ ]:
print(f"Shape del DataFrame delle feature: {X_tsfel.shape}")
np.save('hhar_tsfel.npy', X_tsfel)

Shape del DataFrame delle feature: (46787, 6, 45)
